# Rectilinear mesh benchmark

We want to make sure we get identical results for a rectilinear and regularly-spaced mesh.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from petsc4py import PETSc

%matplotlib inline

import conduction

In [ ]:
minX, maxX = 0., 1.
minY, maxY = 0., 1.
nx, ny, nz = 10, 10, 100


def zfun(z, C=1., k=0.5):
    return C*(1.0 - np.exp(-k*z))

zrange = np.linspace(0, 10, nz)

zaxis = zfun(zrange)
print("min {}, max {}".format(zaxis.min(), zaxis.max()))

plt.plot(zrange, zaxis)

In [ ]:
mesh1.rhs.setValuesBlockedLocal()

In [ ]:
minZ, maxZ = zaxis.min(), zaxis.max()

mesh1 = conduction.Conduction3D((minX, minY, minZ), (maxX, maxY, maxZ), (nx, ny, nz))
mesh2 = conduction.Conduction3D((minX, minY, zrange.min()), (maxX, maxY, zrange.max()), (nx, ny, nz))
mesh2.refine(z_fn=zfun)

print("mesh1 bbox {}".format(mesh1.dm.getBoundingBox()))
print("mesh2 bbox {}".format(mesh2.dm.getBoundingBox()))

## Update material properties

In [ ]:
n = nx*ny*nz

k = np.ones(n)
H = np.zeros(n)

mask = mesh1.coords[:,0] >= 0.5

# k[mask] = 5.

mesh1.update_properties(k, H)
mesh2.update_properties(k, H)

## Boundary conditions

In [ ]:
mesh1.boundary_condition('minZ', 1.0, flux=False)
mesh1.boundary_condition('maxZ', 0.0, flux=False)

mesh2.boundary_condition('minZ', 1.0, flux=False)
mesh2.boundary_condition('maxZ', 0.0, flux=False)
# mesh2.bc['minZ']['val'] *=2

In [ ]:
sol1 = mesh1.solve('bcgs')
sol2 = mesh2.solve('bcgs')

In [ ]:
fig = plt.figure(figsize=(15,8))
ax1 = fig.add_subplot(121, projection='3d', xlim=(minX, maxX), ylim=(minY, maxY),)
sc1 = ax1.scatter3D(mesh1.coords[:,0], mesh1.coords[:,1], mesh1.coords[:,2], c=sol1)
fig.colorbar(sc1)

ax2 = fig.add_subplot(122, projection='3d', xlim=(minX, maxX), ylim=(minY, maxY),)
sc2 = ax2.scatter3D(mesh2.coords[:,0], mesh2.coords[:,1], mesh2.coords[:,2], c=sol2)
fig.colorbar(sc2)


## Calculate heat flux

In [ ]:
Xcoords1 = np.unique(mesh1.coords[:,0])
Ycoords1 = np.unique(mesh1.coords[:,1])
Zcoords1 = np.unique(mesh1.coords[:,2])

Xcoords2 = np.unique(mesh2.coords[:,0])
Ycoords2 = np.unique(mesh2.coords[:,1])
Zcoords2 = np.unique(mesh2.coords[:,2])

dTdx1, dTdy1, dTdz1 = np.gradient(sol1.reshape(nx,ny,nz), Xcoords1, Ycoords1, Zcoords1)
dTdx2, dTdy2, dTdz2 = np.gradient(sol2.reshape(nx,ny,nz), Xcoords2, Ycoords2, Zcoords2)

In [ ]:
from scipy.interpolate import griddata

xq, yq, zq = np.meshgrid(Xcoords1, Ycoords1, Zcoords1)
sol2_grid = griddata(mesh2.coords, sol2, (xq, yq, zq))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,8))

im1 = ax1.imshow(sol1.reshape(nz,ny,nx)[:,5,:], origin='lower', aspect='auto')
fig.colorbar(im1, ax=ax1)

im2 = ax2.imshow(np.rot90(sol2_grid[:,5,:], 3), origin='lower', aspect='auto')
fig.colorbar(im2, ax=ax2)

In [ ]:
def solve(self, solver='bcgs'):
    from petsc4py import PETSc
    matrix = self.mat
    rhs = self.rhs
    res = self.res
    lres = self.lres

    ksp = PETSc.KSP().create()
    ksp.setType(solver)
    ksp.setOperators(matrix)
    # pc = ksp.getPC()
    # pc.setType('gamg')
    ksp.setFromOptions()
    ksp.setTolerances(1e-10, 1e-50)
    ksp.solve(rhs, res)
    # We should hand this back to local vectors
    self.dm.globalToLocal(res, lres)
    return lres.array

In [ ]:
w_mean = mesh2.w.mean(axis=0)
dist = np.linalg.norm(w_mean, axis=1)

In [ ]:
print 1 - dist[545]
diag2 = mesh2.mat.getDiagonal()
diag2.array[mesh2.dirichlet_mask] *= (1.0 - dist)[mesh2.dirichlet_mask]
mesh2.mat.setDiagonal(diag2)

In [ ]:
diag2 = mesh2.mat.getDiagonal()
diag2.array[mesh2.dirichlet_mask] += dist[mesh2.dirichlet_mask]
mesh2.mat.setDiagonal(diag2)

In [ ]:

sol2 = solve(mesh2)


In [ ]:
xx = np.array([-0.5,0,0.5, -0.5,0,0.5, -0.5,0,0.5], dtype=float) + 5
yy = np.array([-1,-1,-1, 0,0,0, 1,1,1], dtype=float) + 5

cxy = np.column_stack([xx, yy])

plt.scatter(xx, yy)

In [ ]:
diff = cxy[5] - cxy
diff.mean(axis=0)

In [ ]:
np.linalg.norm(mesh1.w[:,545].mean(axis=0))

In [ ]:
mesh2.w[:,545].mean(axis=0)

In [ ]:
np.linalg.norm(mesh1.w.mean(axis=0), axis=1)[545]

In [ ]:
mesh1.construct_matrix()
mesh1.cols[:,545]

# np.where(mesh1.dirichlet_mask)

In [ ]:
diag1 = mesh1.mat.getDiagonal()
diag2 = mesh2.mat.getDiagonal()

ddiag = diag1 - diag2

rescale = ddiag.array.reshape(nz,ny,nx)

diag2b = diag2 - 0.1*diag2
mesh2.mat.setDiagonal(diag2b)
# mesh2.mat.view()


def solve(self, solver='bcgs'):
    from petsc4py import PETSc
    matrix = self.mat
    rhs = self.rhs
    res = self.res
    lres = self.lres

    ksp = PETSc.KSP().create()
    ksp.setType(solver)
    ksp.setOperators(matrix)
    # pc = ksp.getPC()
    # pc.setType('gamg')
    ksp.setFromOptions()
    ksp.setTolerances(1e-10, 1e-50)
    ksp.solve(rhs, res)
    # We should hand this back to local vectors
    self.dm.globalToLocal(res, lres)
    return lres.array


sol2 = solve(mesh2)

In [ ]:
fig = plt.figure(figsize=(15,8))
ax1 = fig.add_subplot(121, projection='3d', xlim=(minX, maxX), ylim=(minY, maxY),)
sc1 = ax1.scatter3D(mesh1.coords[:,0], mesh1.coords[:,1], mesh1.coords[:,2], c=dTdy2.ravel())
fig.colorbar(sc1)

ax2 = fig.add_subplot(122, projection='3d', xlim=(minX, maxX), ylim=(minY, maxY),)
sc2 = ax2.scatter3D(mesh2.coords[:,0], mesh2.coords[:,1], mesh2.coords[:,2], c=dTdy2.ravel())
fig.colorbar(sc2)

In [ ]:
print mesh1.bc['minZ']['val']
print mesh2.bc['minZ']['val']

print mesh1.bc['minZ']['delta']
print mesh2.bc['minZ']['delta']

In [ ]:
1./-mesh1.bc['minZ']['delta']

In [ ]:
1./-mesh2.bc['minZ']['delta']

In [ ]:
coords = mesh2.coords

Zcoords = np.unique(coords[:,2])

In [ ]:
Zcoords[1] - Zcoords[0]

In [ ]:
mesh2.bc['minZ']['val'] = 1./-(Zcoords[1] - Zcoords[0])
mesh2.bc['minZ']['val'] *= 2